In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2
from PIL import Image
import time
import glob

#Directory locations for the code
image_path = "Data\\img_align_celeba"

#Cropped Output
output_path = "Data\\cropped_faces"
if not os.path.exists(output_path):
        os.mkdir(output_path)

In [ ]:
#####Cropping just the face of the celebrities#####
face_cascade = "Data\\xml\\haarcascade_frontalface_default.xml"

def crop_and_save_images(cascade, imgname, output_path):
    img = cv2.imread(os.path.join(image_path, imgname))
    for i, face in enumerate(cascade.detectMultiScale(img)):
        #start = time.time()
        x, y, w, h = face
        sub_face = img[y:y + h, x:x + w]
        img_path = os.path.join(output_path, imgname)
        cv2.imwrite(img_path, sub_face)
        #if i%1000 == 0:
            #end = time.time() - start
            #print(i,"took", end, "seconds")
            #print(i, "Cropped")
            #start = time.time()
            
#Cropping the images to contain only faces
cascade = cv2.CascadeClassifier(face_cascade)
for f in [f for f in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, f))]:
    crop_and_save_images(cascade, f, output_path)

In [2]:
#some image manipulation functions
def resizer(path,scale="L",resize_x=28,resize_y=28):
    return np.array(Image.open(path).convert(scale).resize((resize_x,resize_y), Image.ANTIALIAS)).ravel().tolist()

def images_from_dirs(dirs,start,end,scaled = False):
    train_img = 255 - np.array([resizer(i,"L",28,28) for i in glob.glob(dirs)[start:end]],dtype='float32')
    return train_img if scaled else train_img/255.0

In [3]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
 
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=2)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=2)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
no_of_training_data = 20000
training_data_start_index = 0

In [5]:
#Eyeglasses label
feature = pd.read_csv("Data/glasses.csv")
eye_glasses = feature[["Images","Eyeglasses"]]

Y_label = feature.Eyeglasses
Y_label = np.array([i if i == 1 else 0 for i in list(Y_label)])

In [ ]:
#Importing images as numpy objects
images = images_from_dirs(output_path + "\\*.jpg",training_data_start_index,no_of_training_data)
data = {"x":images,"y":Y_label[training_data_start_index:no_of_training_data]}

In [ ]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="tmp/mnist_convnet_model")
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": data["x"]},
    y=data["y"],
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_task_type': 'worker', '_master': '', '_is_chief': True, '_save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001445011D4E0>, '_save_summary_steps': 100, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_model_dir': 'tmp/mnist_convnet_model', '_session_config': None, '_task_id': 0, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_service': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.39879483  0.60120517]
 [ 0.44568247  0.55431753]
 [ 0.45018166  0.5498184 ]
 [ 0.48659316  0.51340687]
 [ 0.46449119  0.53550881]
 [ 0.43981844  0.56018156]
 [ 0.43031105  0.56968904]
 [ 0.45518649  0.54481357]
 [ 0.43386656  0.5661335 ]
 [ 0.4733

INFO:tensorflow:loss = 0.294559, step = 101 (56.211 sec)
INFO:tensorflow:probabilities = [[ 0.8981154   0.10188464]
 [ 0.85239762  0.14760239]
 [ 0.833161    0.16683905]
 [ 0.90209097  0.097909  ]
 [ 0.87660587  0.12339418]
 [ 0.89880019  0.10119987]
 [ 0.81562787  0.18437211]
 [ 0.89428979  0.10571025]
 [ 0.93475133  0.06524872]
 [ 0.91382706  0.08617293]
 [ 0.88524014  0.11475984]
 [ 0.8726036   0.12739637]
 [ 0.88556206  0.11443795]
 [ 0.85215509  0.14784494]
 [ 0.85788876  0.14211124]
 [ 0.88084346  0.11915655]
 [ 0.91308433  0.08691563]
 [ 0.82654643  0.17345354]
 [ 0.91151762  0.08848239]
 [ 0.840424    0.159576  ]
 [ 0.91492474  0.08507525]
 [ 0.93961698  0.06038304]
 [ 0.82779175  0.17220831]
 [ 0.91171831  0.08828174]
 [ 0.92646694  0.07353299]
 [ 0.90044415  0.09955584]
 [ 0.92777014  0.07222992]
 [ 0.8798303   0.12016969]
 [ 0.92329246  0.07670756]
 [ 0.87459755  0.12540247]
 [ 0.8980723   0.10192768]
 [ 0.89095908  0.1090409 ]
 [ 0.91473138  0.08526859]
 [ 0.95123667  0.048

INFO:tensorflow:global_step/sec: 1.81859
INFO:tensorflow:probabilities = [[ 0.95787746  0.04212258]
 [ 0.95489943  0.04510057]
 [ 0.90624797  0.09375206]
 [ 0.91753983  0.08246019]
 [ 0.97728807  0.02271187]
 [ 0.94651538  0.05348457]
 [ 0.95333338  0.04666663]
 [ 0.95350057  0.04649941]
 [ 0.92640948  0.07359055]
 [ 0.94924486  0.05075512]
 [ 0.94417065  0.05582928]
 [ 0.93639016  0.06360982]
 [ 0.88397628  0.11602366]
 [ 0.9360581   0.06394195]
 [ 0.93607664  0.06392328]
 [ 0.96124959  0.03875046]
 [ 0.96641636  0.03358363]
 [ 0.95518821  0.04481176]
 [ 0.91446382  0.08553617]
 [ 0.86426687  0.13573316]
 [ 0.95378858  0.04621137]
 [ 0.94776142  0.05223855]
 [ 0.91418022  0.08581977]
 [ 0.97468013  0.02531991]
 [ 0.93458104  0.06541893]
 [ 0.84044886  0.15955116]
 [ 0.93732429  0.06267573]
 [ 0.88638139  0.11361857]
 [ 0.95335597  0.04664395]
 [ 0.94950229  0.05049771]
 [ 0.83504963  0.16495042]
 [ 0.95890933  0.04109069]
 [ 0.83040255  0.16959742]
 [ 0.89216375  0.10783629]
 [ 0.9047